In [2]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [3]:
token = 'vk1.a.FArh1Tr_1S0qN4q0Hj7NH9JZncGGTRMh9c10sBwldNDIWozpVN0zWxrodn3Xq1UXpir5bmUM1bwUAMKRe0-NRauES5XvYP3K5e76YWoH6cC1fL8OaoYZu1228xmTEknzMRtz4Ly03HBZwrDcCyy3r4ZZdfS3rmlToFXPF9eMSHnuEBrpmC3S9f2jNhzDzTx-LStTkhp-wJCdrKwQMZLZIA'

In [4]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [7]:
vk.messages.send(
    chat_id=1,
    random_id=1,
    message='hello')

0

In [8]:
df = pd.read_csv('ads_data.csv')

In [15]:
ads_data = df \
    .groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'}) \
    .pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [19]:
ads_data = ads_data.fillna(0).assign(ctr=ads_data.click / ads_data.view) # СОЗДАНИЕ КОЛОНКИ

In [23]:
top_ctr = ads_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [24]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [31]:
path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-e-manzhos/lesson_7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '278479522|0|-1|909218|ed5a6907cf|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|ceff1d7d9e4c22c678c228c962bb81aa||||eyJkaXNrIjoiMTkiLCJwZWVyX3NlbmRlciI6Ii0yMjI3ODg2MjcifQ=='}

In [32]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [33]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [34]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='TOP CTR',
    attachment = attachment)

0